### Install Dependencies

In [ ]:
!pip install opencv-python==3.4.2.16
!pip install imutils
!pip install skimage
!pip install matplotlib
!pip install numpy

### Import packages

In [5]:
import cv2
from skimage.measure import compare_ssim
import imutils
import matplotlib.pyplot as plt
import numpy as np
import os
%matplotlib inline

In [ ]:
img = cv2.imread('./NMPS-CD/Road/Ref/Frame1.jpg')
img.shape

In [6]:
ref_dir = '../NMPS-CD/Road/Ref'
video_dir = '../NMPS-CD/Road/Track1'
#video_dir = '../NMPS-CD/Road/Track2'
ref_data = []
video_data = []
n_ref_frames=0
n_video_frames=0
i=1
for f1 in os.listdir(ref_dir):
    img = cv2.imread(os.path.join(ref_dir,'Frame'+str(i)+'.jpg'))
    img = cv2.resize(img, (720,1280), interpolation = cv2.INTER_AREA)
    ref_data.append(img)
    n_ref_frames+=1
    i+=1
i=1
for f1 in os.listdir(video_dir):
    img = cv2.imread(os.path.join(video_dir,'Frame'+str(i)+'.jpg'))
    img = cv2.resize(img, (720,1280), interpolation = cv2.INTER_AREA)
    video_data.append(img)
    n_video_frames+=1
    i+=1

In [ ]:
def getHomography(kpsA, kpsB, featuresA, featuresB, matches, reprojThresh):
    # convert the keypoints to numpy arrays
    kpsA = np.float32([kp.pt for kp in kpsA])
    kpsB = np.float32([kp.pt for kp in kpsB])
    
    if len(matches) > 4:

        # construct the two sets of points
        ptsA = np.float32([kpsA[m.queryIdx] for m in matches])
        ptsB = np.float32([kpsB[m.trainIdx] for m in matches])
        
        # estimate the homography between the sets of points
        (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC,
            reprojThresh)
        return (matches, H, status)
    else:
        return None

In [ ]:
from math import sqrt

trainImg=cv2.imread('./NMPS-CD/Road/Ref/Frame1.jpg')
trainImg_gray=cv2.imread('./NMPS-CD/Road/Ref/Frame1.jpg', cv2.IMREAD_GRAYSCALE)
queryImg=cv2.imread('./NMPS-CD/Road/Track1/Frame10.jpg')
queryImg_gray=cv2.imread('./NMPS-CD/Road/Track1/Frame10.jpg', cv2.IMREAD_GRAYSCALE)

#Detect keypoints and compute descriptors using AKAZE
descriptor = cv2.ORB_create()
descriptor = cv2.AKAZE_create()
kpsA, featuresA = descriptor.detectAndCompute(trainImg_gray, None)
kpsB, featuresB = descriptor.detectAndCompute(queryImg_gray, None)

#Use brute-force matcher to find 2-nn matches
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
nn_matches = bf.knnMatch(featuresA, featuresB, 2)

# loop over the raw matches
matches = []
ratio=0.4
for m, n in nn_matches:
    if m.distance < ratio * n.distance:
        matches.append(m)
img3 = cv2.drawMatches(trainImg,kpsA,queryImg,kpsB,np.random.choice(matches,100),
                           None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img3)
plt.show()

cv2.namedWindow("img", cv2.WINDOW_NORMAL)
cv2.imshow("img",cv2.resize(img3, (960, 540)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
M = getHomography(kpsA, kpsB, featuresA, featuresB, matches, reprojThresh=4)
if M is None:
    print("Error!")
(matches, H, status) = M
print(H)
print(matches[0:4])

In [ ]:
# Apply panorama correction
width = trainImg.shape[1] + queryImg.shape[1]
height = trainImg.shape[0] + queryImg.shape[0]

result = cv2.warpPerspective(trainImg, H, (width, height))
#result[0:(queryImg.shape[0]+0), 0:(queryImg.shape[1]+0)] = queryImg
(score, diff) = compare_ssim(trainImg_gray,cv2.cvtColor(result[0:(queryImg.shape[0]+0), 0:(queryImg.shape[1]+0)], cv2.COLOR_BGR2GRAY),full=True)
diff = (diff * 255).astype("uint8")
thresh = cv2.threshold(diff, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

cv2.namedWindow("thresh", cv2.WINDOW_NORMAL)
cv2.imshow("thresh", thresh)
plt.figure(figsize=(20,10))
plt.imshow(result)

plt.axis('off')
plt.show()
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
import cv2 
import numpy as np 

#out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (1280,720))
#blurw = cv2.VideoWriter('outpy_blur,.avi',cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (1280,720))
ref_ptr=0

for i in range(0,n_video_frames):
    # Open the image files. 
    img1_color = video_data[i]  # Image to be aligned. 
    img2_color = None    # Reference image. 
    matches=0
    kp1,d1=None,None
    kp2,d2=None,None
    while(True):
    # Convert to grayscale. 
        img2_color=ref_data[ref_ptr]
        img1 = cv2.cvtColor(img1_color, cv2.COLOR_BGR2GRAY) 
        img2 = cv2.cvtColor(img2_color, cv2.COLOR_BGR2GRAY) 
        img2_prev = None
        img2_fwd = None
        if(ref_ptr>5):
            img2_prev = cv2.cvtColor(ref_data[ref_ptr-5], cv2.COLOR_BGR2GRAY)
        else:
            img2_prev = cv2.cvtColor(ref_data[ref_ptr], cv2.COLOR_BGR2GRAY)
        if(ref_ptr<len(ref_data)-5):
            img2_fwd = cv2.cvtColor(ref_data[ref_ptr+5], cv2.COLOR_BGR2GRAY)
        else:
            img2_fwd = cv2.cvtColor(ref_data[ref_ptr], cv2.COLOR_BGR2GRAY)
        height, width = img2.shape 

        # Create ORB detector with 5000 features. 
        orb_detector = cv2.ORB_create(5000) 
        orb_detector_prev = cv2.ORB_create(5000)
        orb_detector_fwd = cv2.ORB_create(5000)
        # Find keypoints and descriptors. 
        # The first arg is the image, second arg is the mask 
        #  (which is not reqiured in this case). 
        kp1, d1 = orb_detector.detectAndCompute(img1, None) 
        kp2, d2 = orb_detector.detectAndCompute(img2, None) 
        
        kp3, d3 = orb_detector_prev.detectAndCompute(img1, None) 
        kp4, d4 = orb_detector_prev.detectAndCompute(img2_prev, None)
        
        kp5, d5 = orb_detector.detectAndCompute(img1, None) 
        kp6, d6 = orb_detector.detectAndCompute(img2_fwd, None)

        # Match features between the two images. 
        # We create a Brute Force matcher with  
        # Hamming distance as measurement mode. 
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True) 

        # Match the two sets of descriptors. 
        matches = matcher.match(d1, d2) 
        matches_prev = matcher.match(d3, d4) 
        matches_fwd=matcher.match(d5, d6) 
        
        if(max(len(matches),len(matches_prev),len(matches_fwd)) == len(matches)):
            while(True):
                # Convert to grayscale. 
                img2_color=ref_data[ref_ptr] 
                img2 = cv2.cvtColor(img2_color, cv2.COLOR_BGR2GRAY) 
                img2_prev = None
                img2_fwd = None
                if(ref_ptr>1):
                    img2_prev = cv2.cvtColor(ref_data[ref_ptr-1], cv2.COLOR_BGR2GRAY)
                else:
                    img2_prev = cv2.cvtColor(ref_data[ref_ptr], cv2.COLOR_BGR2GRAY)
                if(ref_ptr<len(ref_data)-1):
                    img2_fwd = cv2.cvtColor(ref_data[ref_ptr+1], cv2.COLOR_BGR2GRAY)
                else:
                    img2_fwd = cv2.cvtColor(ref_data[ref_ptr], cv2.COLOR_BGR2GRAY)
                height, width = img2.shape 

                # Create ORB detector with 5000 features. 
                orb_detector = cv2.ORB_create(5000) 
                orb_detector_prev = cv2.ORB_create(5000)
                orb_detector_fwd = cv2.ORB_create(5000)
                # Find keypoints and descriptors. 
                # The first arg is the image, second arg is the mask 
                #  (which is not reqiured in this case). 
                kp1, d1 = orb_detector.detectAndCompute(img1, None) 
                kp2, d2 = orb_detector.detectAndCompute(img2, None) 

                kp3, d3 = orb_detector_prev.detectAndCompute(img1, None) 
                kp4, d4 = orb_detector_prev.detectAndCompute(img2_prev, None)

                kp5, d5 = orb_detector.detectAndCompute(img1, None) 
                kp6, d6 = orb_detector.detectAndCompute(img2_fwd, None)

                # Match features between the two images. 
                # We create a Brute Force matcher with  
                # Hamming distance as measurement mode. 
                matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True) 

                # Match the two sets of descriptors. 
                matches = matcher.match(d1, d2) 
                matches_prev = matcher.match(d3, d4) 
                matches_fwd=matcher.match(d5, d6) 

                if(max(len(matches),len(matches_prev),len(matches_fwd)) == len(matches)):
                    break
                elif(max(len(matches),len(matches_prev),len(matches_fwd)) == len(matches_prev)):
                    ref_ptr-=1
                    continue
                else:
                    ref_ptr+=1
                    continue
            break
        elif(max(len(matches),len(matches_prev),len(matches_fwd)) == len(matches_prev)):
            ref_ptr-=5
            continue
        else:
            ref_ptr+=5
            continue
    # Sort matches on the basis of their Hamming distance. 
    matches.sort(key = lambda x: x.distance) 

    # Take the top 90 % matches forward. 
    matches = matches[:int(len(matches)*90)] 
    no_of_matches = len(matches) 

    # Define empty matrices of shape no_of_matches * 2. 
    p1 = np.zeros((no_of_matches, 2)) 
    p2 = np.zeros((no_of_matches, 2)) 

    for i in range(len(matches)): 
      p1[i, :] = kp1[matches[i].queryIdx].pt 
      p2[i, :] = kp2[matches[i].trainIdx].pt 

    # Find the homography matrix. 
    homography, mask = cv2.findHomography(p1, p2, cv2.RANSAC) 

    # Use this matrix to transform the 
    # colored image wrt the reference image. 
    transformed_img = cv2.warpPerspective(img1_color, 
                        homography, (width, height)) 

    (score, diff) = compare_ssim(img2,cv2.cvtColor(transformed_img, cv2.COLOR_BGR2GRAY), full=True)
    diff = (diff * 255).astype("uint8")
    thresh = cv2.threshold(diff, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    #kernel = np.ones((5,5),np.uint8)
    #erosion = cv2.erode(diff,kernel,iterations = 1)
    #blur=cv2.GaussianBlur(thresh, (15,15),0)
    #blur=cv2.medianBlur(thresh,15)
    #kernel=np.zeros((5,5), np.uint8)
    #erosion = cv2.dilate(blur, kernel, iterations=1)
    
    cv2.namedWindow("thresh", cv2.WINDOW_NORMAL)
    cv2.imshow("thresh", thresh)
    cv2.imwrite('/NMPS-CD/Road/Output1'+str(i)+'.jpg',cv2.cvtColor(thresh,cv2.COLOR_GRAY2RGB))
    #out.write(cv2.cvtColor(thresh,cv2.COLOR_GRAY2RGB))
    #cv2.namedWindow("blur", cv2.WINDOW_NORMAL)
    #cv2.imshow("blur", blur)
    #blurw.write(cv2.cvtColor(blur,cv2.COLOR_GRAY2RGB))
    
    #cv2.namedWindow("erosion", cv2.WINDOW_NORMAL)
    #cv2.imshow("erosion", erosion)
    
    #cv2.namedWindow("ref.jpg", cv2.WINDOW_NORMAL)
    #cv2.namedWindow("align.jpg", cv2.WINDOW_NORMAL)
    #cv2.imshow('ref.jpg', img2_color)
    #cv2.imshow('align.jpg', img1_color)
    keyboard = cv2.waitKey(30) & 0xFF
    if keyboard == 'q' or keyboard == 27:
        break
#out.release()
#blurw.release()
cv2.destroyAllWindows()